In [ ]:
import json
import pandas as pd
import numpy as np
from portstress.core.vol_models.wing_model import WingModel

pd.options.display.float_format = '{:,.4f}'.format
pd.set_option('display.max_rows', None)  # Set max_rows to None to display all rows
pd.set_option('display.max_columns', None)  # Set max_columns to None to display all columns


In [2]:


# Create synthetic implied vol market data
strikes = np.linspace(60, 1400, 17)  # More strikes
true_params = dict(vr=0.20, sr=-0.1, pc=1.5, cc=0.8, dc=-0.3, uc=0.3, dsm=0.4, usm=0.4)
F = 1000
Ref = 1000
ATM = 1000

# Generate vols from the wing model with some noise
_, vols = WingModel.wing_vol_curve_with_smoothing(
    strikes=strikes,
    F=F,
    vr=true_params['vr'],
    sr=true_params['sr'],
    pc=true_params['pc'],
    cc=true_params['cc'],
    dc=true_params['dc'],
    uc=true_params['uc'],
    dsm=true_params['dsm'],
    usm=true_params['usm'],
    VCR=0,
    SCR=0,
    SSR=100,
    Ref=Ref,
    ATM=ATM
)
noisy_vols = vols + np.random.normal(0, 0.005, size=len(vols))  # Add small noise

# Create DataFrame and load into tool
df = pd.DataFrame({'strike': strikes, 'vol': noisy_vols})

wm_ui = WingModel()
wm_ui.set_market_data(df['strike'].values, df['vol'].values)
wm_ui.create_interactive_wing_fit_ui()

Button(button_style='success', description='Fit to Market Data', style=ButtonStyle())

Output()

In [ ]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize
import plotly.graph_objs as go
from ipywidgets import (
    FloatSlider, FloatText, Layout, HBox, VBox,
    interactive_output, Button
)

# --- Wing vol curve model ---
def wing_vol_curve_with_smoothing(
    strikes,
    F,              # Forward price used to define log-moneyness
    vr,             # Volatility reference at central skew point
    sr,             # Slope reference at central skew point
    pc,             # Put curvature: bend on put wing (left of ATM)
    cc,             # Call curvature: bend on call wing (right of ATM)
    dc,             # Down cutoff (log-moneyness, negative value)
    uc,             # Up cutoff (log-moneyness, positive value)
    dsm=0.5,        # Down smoothing range beyond dc
    usm=0.5,        # Up smoothing range beyond uc
    VCR=0.0,        # Volatility change rate w.r.t forward move
    SCR=0.0,        # Slope change rate w.r.t forward move
    SSR=100,        # Skew swimmingness rate (0 = fixed, 100 = ATM follows F)
    Ref=None,       # Reference price (for fixed skew)
    ATM=None        # ATM forward price (for floating skew)
):
    """
    Generate a volatility curve using the Orc Wing Model with parabolic smoothing.

    This version reproduces Orc's exact smoothing style:
    - Parabolic skew on both put and call wings.
    - Smooth cubic (Hermite-style) interpolation in the smoothing zones.
    - Flat extrapolation beyond the smoothing cutoffs.

    Parameters:
        strikes (array): Strike prices to compute volatility for.
        F (float): Forward price for normalization (used to compute log-moneyness).
        vr (float): Volatility reference at the central skew point.
        sr (float): Slope reference at the central skew point.
        pc (float): Curvature for the put wing (left of ATM).
        cc (float): Curvature for the call wing (right of ATM).
        dc (float): Down cutoff in log-moneyness where put wing ends.
        uc (float): Up cutoff in log-moneyness where call wing ends.
        dsm (float): Down smoothing width beyond dc (default 0.5).
        usm (float): Up smoothing width beyond uc (default 0.5).
        VCR (float): Volatility change rate when forward deviates from reference.
        SCR (float): Slope change rate when forward deviates from reference.
        SSR (float): Skew swimmingness rate (0 = fixed skew, 100 = ATM anchored).
        Ref (float): Reference forward price.
        ATM (float): ATM forward used in floating skew calculation.

    Returns:
        strikes (array): The original strike array.
        vols (array): Computed implied volatilities corresponding to the strikes.
    """
    ssr_frac = SSR / 100.0
    F_eff = F if SSR == 100 else (Ref if SSR == 0 else (1 - ssr_frac) * Ref + ssr_frac * ATM)
    vc = vr - VCR * ssr_frac * (F_eff - Ref) / Ref
    sc = sr - SCR * ssr_frac * (F_eff - Ref) / Ref

    x = np.log(strikes / F_eff)
    vols = np.zeros_like(x)

    xL0, xL1 = dc - dsm, dc
    yL0 = vc + sc * xL0 + pc * xL0**2
    yL1 = vc + sc * xL1 + pc * xL1**2
    dyL0 = sc + 2 * pc * xL0
    dyL1 = sc + 2 * pc * xL1

    xR0, xR1 = uc, uc + usm
    yR0 = vc + sc * xR0 + cc * xR0**2
    yR1 = vc + sc * xR1 + cc * xR1**2
    dyR0 = sc + 2 * cc * xR0
    dyR1 = sc + 2 * cc * xR1

    for i, xi in enumerate(x):
        if xi < xL0:
            vols[i] = yL0 + dyL0 * (xi - xL0)
        elif xL0 <= xi < xL1:
            t = (xi - xL0) / (xL1 - xL0)
            h00, h10 = 2*t**3 - 3*t**2 + 1, t**3 - 2*t**2 + t
            h01, h11 = -2*t**3 + 3*t**2, t**3 - t**2
            vols[i] = h00 * yL0 + h10 * (xL1 - xL0) * dyL0 + h01 * yL1 + h11 * (xL1 - xL0) * dyL1
        elif dc <= xi < 0:
            vols[i] = vc + sc * xi + pc * xi ** 2
        elif 0 <= xi <= uc:
            vols[i] = vc + sc * xi + cc * xi ** 2
        elif xR0 < xi <= xR1:
            t = (xi - xR0) / (xR1 - xR0)
            h00, h10 = 2*t**3 - 3*t**2 + 1, t**3 - 2*t**2 + t
            h01, h11 = -2*t**3 + 3*t**2, t**3 - t**2
            vols[i] = h00 * yR0 + h10 * (xR1 - xR0) * dyR0 + h01 * yR1 + h11 * (xR1 - xR0) * dyR1
        else:
            vols[i] = yR1 + dyR1 * (xi - xR1)

    return strikes, vols

# --- Fit function ---
def fit_wing_model_to_data(
    strikes, market_vols, F_init, Ref_init=None, ATM_init=None,
    fit_vcr_scr_ssr=True, fit_f_ref_atm=False
):
    Ref_init = Ref_init or F_init
    ATM_init = ATM_init or F_init

    x_moneyness = np.log(strikes / F_init)
    weights = 1 / (np.abs(x_moneyness) + 0.05)

    def unpack_params(p):
        idx = 0
        keys = ["vr", "sr", "pc", "cc", "dc", "uc", "dsm", "usm"]
        vals = dict(zip(keys, p[idx:idx+8]))
        idx += 8
        if fit_vcr_scr_ssr:
            vals.update(dict(VCR=p[idx], SCR=p[idx+1], SSR=p[idx+2]))
            idx += 3
        else:
            vals.update(dict(VCR=0.0, SCR=0.0, SSR=100.0))
        if fit_f_ref_atm:
            vals.update(dict(F=p[idx], Ref=p[idx+1], ATM=p[idx+2]))
        else:
            vals.update(dict(F=F_init, Ref=Ref_init, ATM=ATM_init))
        return vals

    def objective(p):
        params = unpack_params(p)
        _, vols = wing_vol_curve_with_smoothing(
            strikes=strikes,
            **params
        )
        return np.mean(weights * (vols - market_vols) ** 2)

    # Initial guess
    p0 = [
        0.2,   # vr
        -0.1,  # sr
        1.5,   # pc
        1.0,   # cc
        -0.3,  # dc
        0.3,   # uc
        0.4,   # dsm
        0.4,   # usm
    ]
    bounds = [
        (0.05, 1.0),   # vr
        (-2.0, 2.0),   # sr
        (0.01, 5.0),   # pc
        (0.01, 5.0),   # cc
        (-1.5, -0.01), # dc
        (0.01, 1.5),   # uc
        (0.01, 2.0),   # dsm
        (0.01, 2.0),   # usm
    ]

    if fit_vcr_scr_ssr:
        p0 += [0.0, 0.0, 100.0]
        bounds += [(-2.0, 2.0), (-2.0, 2.0), (0, 100)]

    if fit_f_ref_atm:
        p0 += [F_init, Ref_init, ATM_init]
        bounds += [(F_init * 0.9, F_init * 1.1)] * 3

    result = minimize(objective, p0, bounds=bounds, method='L-BFGS-B')
    return unpack_params(result.x)

# --- Widget helpers ---
def float_slider_text(label, minv, maxv, step, value):
    slider = FloatSlider(min=minv, max=maxv, step=step, value=value,
                         description=label, continuous_update=False, layout=Layout(width='60%'))
    text = FloatText(value=value, layout=Layout(width='80px'))

    # Corrected two-way linkage
    def on_slider_change(change):
        text.value = change['new']

    def on_text_change(change):
        slider.value = change['new']

    slider.observe(on_slider_change, names='value')
    text.observe(on_text_change, names='value')

    return slider, text

# --- Interactive plot ---

def interactive_wing_plot_with_data(
    vr, sr, pc, cc, dc, uc, dsm, usm, VCR, SCR, SSR, F, Ref, ATM
):
    """
    Plots the Wing Model curve interactively alongside market data (if available).

    Automatically adjusts strike range to fit BTC-like markets (e.g. strikes in 20K–400K).
    """
    # Default strike range
    strike_min, strike_max = 30, 180

    # If market data is attached, use it to expand range
    if hasattr(interactive_wing_plot_with_data, "market_data"):
        mk_strikes, mk_vols = interactive_wing_plot_with_data.market_data
        strike_min = np.min(mk_strikes)  # avoid near-zero issues
        strike_max = np.max(mk_strikes)
    else:
        mk_strikes, mk_vols = [], []

    # Generate strike grid
    strikes = np.linspace(strike_min, strike_max, 300)

    # Compute model vols
    _, vols = wing_vol_curve_with_smoothing(
        strikes, F, vr, sr, pc, cc, dc, uc, dsm, usm,
        VCR, SCR, SSR, Ref, ATM
    )

    # Build Plotly figure
    fig = go.Figure()

    # Add model vol curve
    fig.add_trace(go.Scatter(
        x=strikes, y=vols,
        mode='lines',
        name="Wing Model Vol",
        line=dict(color="blue", width=2)
    ))

    # Add market data points if available
    if len(mk_strikes) > 0:
        fig.add_trace(go.Scatter(
            x=mk_strikes, y=mk_vols,
            mode='markers',
            name="Market Data",
            marker=dict(color="red", size=6, symbol="circle")
        ))

    fig.update_layout(
        title="Wing Volatility Curve Fit (Interactive)",
        xaxis_title="Strike",
        yaxis_title="Implied Volatility",
        template="plotly_white",
        hovermode="x unified",
        height=500,
        legend=dict(x=0.02, y=0.98)
    )

    fig.show()

def set_market_data(strikes, vols):
    interactive_wing_plot_with_data.market_data = (strikes, vols)

# --- Fit UI ---
def create_interactive_wing_fit_ui():
    defs = {
        'vr': ("Vol Ref", 0.05, 0.5, 0.01, 0.20),
        'sr': ("Slope", -2.0, 2.0, 0.1, 0.0),
        'pc': ("Put Curv", 0.0, 5.0, 0.1, 2.0),
        'cc': ("Call Curv", 0.0, 5.0, 0.1, 1.0),
        'dc': ("Down Cut", -1.0, 0.0, 0.05, -0.2),
        'uc': ("Up Cut", 0.0, 1.0, 0.05, 0.2),
        'dsm': ("Down Sm", 0.0, 2.0, 0.1, 0.5),
        'usm': ("Up Sm", 0.0, 2.0, 0.1, 0.5),
        'VCR': ("VCR", -2.0, 2.0, 0.1, 0.0),
        'SCR': ("SCR", -2.0, 2.0, 0.1, 0.0),
        'SSR': ("SSR", 0, 100, 1, 100),
        'F': ("Forward", 80, 120, 1, 100),
        'Ref': ("Ref", 80, 120, 1, 100),
        'ATM': ("ATM", 80, 120, 1, 100),
    }

    # If market data is attached, use it to expand range
    if hasattr(interactive_wing_plot_with_data, "market_data"):
        mk_strikes, _ = interactive_wing_plot_with_data.market_data
        strike_min = np.min(mk_strikes)  # avoid near-zero issues
        strike_max = np.max(mk_strikes)
        defs['F'] = ("Forward", strike_min, strike_max, 1, 100)
        defs['Ref'] = ("Ref", strike_min, strike_max, 1, 100)
        defs['ATM'] = ("ATM", strike_min, strike_max, 1, 100)

    global param_widgets
    param_widgets = {}
    widget_boxes = []

    for k, v in defs.items():
        s, t = float_slider_text(*v)
        param_widgets[k] = (s, t)
        widget_boxes.append(HBox([s, t]))

    plot_out = interactive_output(interactive_wing_plot_with_data, {k: v[0] for k, v in param_widgets.items()})
    fit_button = Button(description="Fit to Market Data", button_style="success")

    def on_fit_click(b):
        if not hasattr(interactive_wing_plot_with_data, "market_data"):
            print("⚠️ Market data not loaded!")
            return

        mk_strikes, mk_vols = interactive_wing_plot_with_data.market_data
        F = param_widgets['F'][0].value
        Ref = param_widgets['Ref'][0].value
        ATM = param_widgets['ATM'][0].value

        fitted = fit_wing_model_to_data(mk_strikes, mk_vols, F, Ref, ATM)
        for k, v in fitted.items():
            param_widgets[k][0].value = v
            param_widgets[k][1].value = v

    fit_button.on_click(on_fit_click)
    display(fit_button, VBox(widget_boxes), plot_out)


In [24]:
import pandas as pd
import numpy as np

# Create synthetic implied vol market data
strikes = np.linspace(60, 140, 17)  # More strikes
true_params = dict(vr=0.20, sr=-0.1, pc=1.5, cc=0.8, dc=-0.3, uc=0.3, dsm=0.4, usm=0.4)
F = 100
Ref = 100
ATM = 100

# Generate vols from the wing model with some noise
_, vols = wing_vol_curve_with_smoothing(
    strikes=strikes,
    F=F,
    vr=true_params['vr'],
    sr=true_params['sr'],
    pc=true_params['pc'],
    cc=true_params['cc'],
    dc=true_params['dc'],
    uc=true_params['uc'],
    dsm=true_params['dsm'],
    usm=true_params['usm'],
    VCR=0,
    SCR=0,
    SSR=100,
    Ref=Ref,
    ATM=ATM
)
noisy_vols = vols + np.random.normal(0, 0.005, size=len(vols))  # Add small noise

# Create DataFrame and load into tool
df = pd.DataFrame({'strike': strikes, 'vol': noisy_vols})
set_market_data(df['strike'].values, df['vol'].values)
create_interactive_wing_fit_ui()



Button(button_style='success', description='Fit to Market Data', style=ButtonStyle())

Output()

In [2]:
def wing_vol_curve_orc_exact(
    strikes,
    F,
    vr,
    sr,
    pc,
    cc,
    dc,
    uc,
    dsm=0.5,
    usm=0.5,
    VCR=0.0,
    SCR=0.0,
    SSR=100,
    Ref=None,
    ATM=None
):
    """
    Exact implementation of the Orc Wing Model based on original Orc formulas.
    
    This version:
    - Follows the Orc-style six-segment structure.
    - Applies exact smoothing formulas derived from Orc documentation.
    - Ensures full continuity at boundaries and quadratic-parabolic smoothing.

    Parameters:
        strikes (array): Strike prices.
        F (float): Forward price for normalization.
        vr (float): Vol reference at the central point.
        sr (float): Slope reference at the central point.
        pc (float): Put curvature.
        cc (float): Call curvature.
        dc (float): Down cutoff (log-moneyness).
        uc (float): Up cutoff (log-moneyness).
        dsm (float): Down smoothing width.
        usm (float): Up smoothing width.
        VCR (float): Vol sensitivity to forward changes.
        SCR (float): Slope sensitivity to forward changes.
        SSR (float): Skew swimmingness rate (0 to 100).
        Ref (float): Reference forward.
        ATM (float): ATM forward.

    Returns:
        strikes (array), vols (array): Volatility curve at given strikes.
    """
    ssr_frac = SSR / 100.0

    if SSR == 0:
        F_eff = Ref
    elif SSR == 100:
        F_eff = ATM
    else:
        F_eff = (1 - ssr_frac) * Ref + ssr_frac * ATM

    # Center vol and slope
    vc = vr - VCR * ssr_frac * (F_eff - Ref) / Ref
    sc = sr - SCR * ssr_frac * (F_eff - Ref) / Ref

    # Log-moneyness x = ln(K/F)
    x = np.log(strikes / F_eff)
    vols = np.zeros_like(x)

    for i, xi in enumerate(x):
        if xi < dc:
            if xi < dc - dsm:
                # Constant extrapolated vol (leftmost flat)
                vols[i] = vc + sc * dc + pc * dc ** 2
            else:
                # Exact Orc down smoothing zone (quadratic blend)
                z = (xi - (dc - dsm)) / dsm  # z in [0, 1]
                a = vc + sc * (dc - dsm) + pc * (dc - dsm) ** 2
                b = vc + sc * dc + pc * dc ** 2
                vols[i] = a + (b - a) * (3 * z ** 2 - 2 * z ** 3)

        elif dc <= xi < 0:
            # Put wing: parabola
            vols[i] = vc + sc * xi + pc * xi ** 2

        elif 0 <= xi <= uc:
            # Call wing: parabola
            vols[i] = vc + sc * xi + cc * xi ** 2

        elif uc < xi <= uc + usm:
            # Exact Orc up smoothing zone
            z = (xi - uc) / usm  # z in [0, 1]
            a = vc + sc * uc + cc * uc ** 2
            b = vc + sc * (uc + usm) + cc * (uc + usm) ** 2
            vols[i] = a + (b - a) * (3 * z ** 2 - 2 * z ** 3)

        else:
            # Constant extrapolated vol (rightmost flat)
            vols[i] = vc + sc * (uc + usm) + cc * (uc + usm) ** 2

    return strikes, vols


In [3]:
def wing_vol_curve_with_smoothing(
    strikes,
    F,              # Forward price used to define log-moneyness
    vr,             # Volatility reference at central skew point
    sr,             # Slope reference at central skew point
    pc,             # Put curvature: bend on put wing (left of ATM)
    cc,             # Call curvature: bend on call wing (right of ATM)
    dc,             # Down cutoff (log-moneyness, negative value)
    uc,             # Up cutoff (log-moneyness, positive value)
    dsm=0.5,        # Down smoothing range beyond dc
    usm=0.5,        # Up smoothing range beyond uc
    VCR=0.0,        # Volatility change rate w.r.t forward move
    SCR=0.0,        # Slope change rate w.r.t forward move
    SSR=100,        # Skew swimmingness rate (0 = fixed, 100 = ATM follows F)
    Ref=None,       # Reference price (for fixed skew)
    ATM=None        # ATM forward price (for floating skew)
):
    """
    Generate a volatility curve using the Orc Wing Model with parabolic smoothing.

    This version reproduces Orc's exact smoothing style:
    - Parabolic skew on both put and call wings.
    - Smooth cubic (Hermite-style) interpolation in the smoothing zones.
    - Flat extrapolation beyond the smoothing cutoffs.

    Parameters:
        strikes (array): Strike prices to compute volatility for.
        F (float): Forward price for normalization (used to compute log-moneyness).
        vr (float): Volatility reference at the central skew point.
        sr (float): Slope reference at the central skew point.
        pc (float): Curvature for the put wing (left of ATM).
        cc (float): Curvature for the call wing (right of ATM).
        dc (float): Down cutoff in log-moneyness where put wing ends.
        uc (float): Up cutoff in log-moneyness where call wing ends.
        dsm (float): Down smoothing width beyond dc (default 0.5).
        usm (float): Up smoothing width beyond uc (default 0.5).
        VCR (float): Volatility change rate when forward deviates from reference.
        SCR (float): Slope change rate when forward deviates from reference.
        SSR (float): Skew swimmingness rate (0 = fixed skew, 100 = ATM anchored).
        Ref (float): Reference forward price.
        ATM (float): ATM forward used in floating skew calculation.

    Returns:
        strikes (array): The original strike array.
        vols (array): Computed implied volatilities corresponding to the strikes.
    """
    # Convert SSR to a fractional weight (0 to 1)
    ssr_frac = SSR / 100.0

    # Compute effective forward (F_eff) based on SSR blend between Ref and ATM
    if SSR == 0:
        F_eff = Ref
    elif SSR == 100:
        F_eff = ATM
    else:
        F_eff = (1 - ssr_frac) * Ref + ssr_frac * ATM

    # Central volatility adjusted by forward deviation from reference
    vc = vr - VCR * ssr_frac * (F_eff - Ref) / Ref

    # Central slope adjusted by forward deviation
    sc = sr - SCR * ssr_frac * (F_eff - Ref) / Ref

    # Convert strike to log-moneyness x = ln(K/F_eff)
    x = np.log(strikes / F_eff)
    vols = np.zeros_like(x)

    # Precompute Hermite endpoints
    xL0, xL1 = dc - dsm, dc
    yL0 = vc + sc * xL0 + pc * xL0 ** 2
    yL1 = vc + sc * xL1 + pc * xL1 ** 2
    dyL0 = sc + 2 * pc * xL0
    dyL1 = sc + 2 * pc * xL1

    xR0, xR1 = uc, uc + usm
    yR0 = vc + sc * xR0 + cc * xR0 ** 2
    yR1 = vc + sc * xR1 + cc * xR1 ** 2
    dyR0 = sc + 2 * cc * xR0
    dyR1 = sc + 2 * cc * xR1

    for i, xi in enumerate(x):
        if xi < xL0:
            # Extrapolate left of smoothing using tangent at xL0
            vols[i] = yL0 + dyL0 * (xi - xL0)

        elif xL0 <= xi < xL1:
            # Hermite interpolation in down smoothing zone
            t = (xi - xL0) / (xL1 - xL0)
            h00 = 2 * t**3 - 3 * t**2 + 1
            h10 = t**3 - 2 * t**2 + t
            h01 = -2 * t**3 + 3 * t**2
            h11 = t**3 - t**2
            vols[i] = h00 * yL0 + h10 * (xL1 - xL0) * dyL0 + h01 * yL1 + h11 * (xL1 - xL0) * dyL1

        elif dc <= xi < 0:
            # Put wing
            vols[i] = vc + sc * xi + pc * xi ** 2

        elif 0 <= xi <= uc:
            # Call wing
            vols[i] = vc + sc * xi + cc * xi ** 2

        elif xR0 < xi <= xR1:
            # Hermite interpolation in up smoothing zone
            t = (xi - xR0) / (xR1 - xR0)
            h00 = 2 * t**3 - 3 * t**2 + 1
            h10 = t**3 - 2 * t**2 + t
            h01 = -2 * t**3 + 3 * t**2
            h11 = t**3 - t**2
            vols[i] = h00 * yR0 + h10 * (xR1 - xR0) * dyR0 + h01 * yR1 + h11 * (xR1 - xR0) * dyR1

        elif xi > xR1:
            # Extrapolate right of smoothing using tangent at xR1
            vols[i] = yR1 + dyR1 * (xi - xR1)

    return strikes, vols

In [4]:
import numpy as np
import plotly.graph_objects as go
from ipywidgets import interact, FloatSlider, FloatText, HBox, VBox, interactive_output, Layout

# Define float slider + input box pair
def float_slider_text(label, min_val, max_val, step, value):
    slider = FloatSlider(min=min_val, max=max_val, step=step, value=value,
                         description=label, continuous_update=False, layout=Layout(width='60%'))
    text = FloatText(value=value, layout=Layout(width='80px'))
    # Link both ways
    def link_slider_text(*args): slider.value = text.value
    def link_text_slider(*args): text.value = slider.value
    slider.observe(link_text_slider, 'value')
    text.observe(link_slider_text, 'value')
    return slider, text

# Plot function
def interactive_wing_plot(
    vr, sr, pc, cc, dc, uc, dsm, usm, VCR, SCR, SSR,
    F, Ref, ATM
):
    strikes = np.linspace(30, 180, 200)
    strikes_out, vols_out = wing_vol_curve_with_smoothing(
        strikes=strikes,
        F=F, vr=vr, sr=sr, pc=pc, cc=cc,
        dc=dc, uc=uc, dsm=dsm, usm=usm,
        VCR=VCR, SCR=SCR, SSR=SSR,
        Ref=Ref, ATM=ATM
    )

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=strikes_out, y=vols_out, mode='lines', name="Implied Vol"))
    fig.update_layout(
        title="Interactive Wing Model Volatility Curve",
        xaxis_title="Strike Price",
        yaxis_title="Implied Volatility",
        template="plotly_white",
        height=500
    )
    fig.show()

# Define all widgets
params = {
    'vr': float_slider_text("Vol Ref", 0.05, 0.5, 0.01, 0.20),
    'sr': float_slider_text("Slope", -2.0, 2.0, 0.1, 0.0),
    'pc': float_slider_text("Put Curv", 0.0, 5.0, 0.1, 2.0),
    'cc': float_slider_text("Call Curv", 0.0, 5.0, 0.1, 1.0),
    'dc': float_slider_text("Down Cut", -1.0, 0.0, 0.05, -0.2),
    'uc': float_slider_text("Up Cut", 0.0, 1.0, 0.05, 0.2),
    'dsm': float_slider_text("Down Sm", 0.0, 2.0, 0.1, 0.5),
    'usm': float_slider_text("Up Sm", 0.0, 2.0, 0.1, 0.5),
    'VCR': float_slider_text("VCR", -2.0, 2.0, 0.1, 0.0),
    'SCR': float_slider_text("SCR", -2.0, 2.0, 0.1, 0.0),
    'SSR': float_slider_text("SSR", 0, 100, 1, 50),
    'F': float_slider_text("Forward F", 80, 120, 1, 100),
    'Ref': float_slider_text("Reference", 80, 120, 1, 90),
    'ATM': float_slider_text("ATM", 80, 120, 1, 110),
}

# Organize layout
widget_boxes = [HBox([s, t]) for s, t in params.values()]
ui = VBox(widget_boxes)

# Create plot output
interactive_plot = interactive_output(interactive_wing_plot, {k: v[0] for k, v in params.items()})

# Display in notebook
display(ui, interactive_plot)


Output()